# M01. Batted Balls

### Imports

In [ ]:
%run "C:\Users\james\Documents\MLB\Code\U1. Imports.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U2. Functions.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U4. Datasets.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U5. Models.ipynb"

### Data

##### MLB Stats API

In [ ]:
start_year, end_year = 2015, 2025

Merge MLB Stats API and Statcast data

In [ ]:
%%time
df = merge_datasets(start_year, end_year)
df = clean_weather(df)
df = create_events(df)
df = create_variables(df)
df = start_data(df)

In [ ]:
api_keep_list = ['date', 'batter', 'pitcher', 'eventsModel', 'launch_angle', 'launch_speed', 'to_l', 'to_lc', 'to_c', 'to_rc', 'to_r', 'b_L'] + ['bb', 'hbp', 'so']

In [ ]:
df = df[api_keep_list]

Note:
- if y > 198.27 and x < 125.42), it's actually to left
- if y > 198.27 and x > 125.42), it's actually to right

Convert data types

In [ ]:
df['proj_date'] = pd.to_datetime(df['date'], format='%Y%m%d')

In [ ]:
df['mlbamid'] = df['batter'].astype(str)

##### Baserunning

In [ ]:
steamer_hitters_df = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_hitters_weekly_log.csv"), encoding='iso-8859-1', usecols=['proj_date', 'mlbamid', 'PA', 'UBR'], dtype='str')

Convert data types

In [ ]:
steamer_hitters_df[['PA', 'UBR']] = steamer_hitters_df[['PA', 'UBR']].astype(float)

In [ ]:
steamer_hitters_df['proj_date'] = pd.to_datetime(steamer_hitters_df['proj_date'])

Calculate UBR per 600 Plate Appearances

In [ ]:
steamer_hitters_df['UBR600'] = steamer_hitters_df['UBR'] / steamer_hitters_df['PA'] * 600

##### Merge

Baserunning Data

In [ ]:
complete_dataset = pd.merge_asof(
    df.sort_values('proj_date'),
    steamer_hitters_df.sort_values('proj_date'),
    by='mlbamid',
    on='proj_date',
    direction='backward'
)

In [ ]:
complete_dataset.shape

##### Sample

Sent launch data to 0 if not batted

In [ ]:
complete_dataset[['launch_angle', 'launch_speed']] = complete_dataset[['launch_angle', 'launch_speed']].fillna(0)

Remove atypical events

In [ ]:
complete_dataset = complete_dataset[~complete_dataset['eventsModel'].isin(["Cut"])].copy()

Select inputs

In [ ]:
batted_ball_inputs = ['launch_angle', 'launch_speed', 'to_l', 'to_lc', 'to_c', 'to_rc', 'to_r', 'b_L', 'UBR600'] + ['bb', 'hbp', 'so']

In [ ]:
complete_dataset = complete_dataset.dropna(subset=batted_ball_inputs)

### Model #1. Expected Outcome

Probability of events given how the baseball was launched, where it was launched to, and some information about the batter, including handedness and base running. Notably excluded park and weather.

$ \hat{\text{eventsModel}} = launch\_angle + launch\_speed + to\_l + to\_lc + to\_c + to\_rc + to\_r + b\_L + UBR600 $

Define model input and outputs

In [ ]:
X = complete_dataset[batted_ball_inputs].values
y = complete_dataset[['eventsModel']].values

##### Encode

In [ ]:
# Create one-hot encoder
encode_outcome = OneHotEncoder(sparse_output=False)
# Fit and transform
y_encoded = encode_outcome.fit_transform(y)
# Create folder
os.makedirs(os.path.join(model_path, "M01. Batted Balls", todaysdate), exist_ok=True)   
# Save
pickle.dump(encode_outcome, open(os.path.join(model_path, "M01. Batted Balls", todaysdate, "encode_output.pkl"), 'wb'))

##### Scale

In [ ]:
# Create scaler
scale_inputs = StandardScaler()
# Fit and transform
X_scaled = scale_inputs.fit_transform(X)
# Save
pickle.dump(scale_inputs, open(os.path.join(model_path, "M01. Batted Balls", todaysdate, "scale_inputs.pkl"), 'wb'))

##### Train

In [ ]:
%%time
# Create neural network
predict_batted_balls = Sequential([
    Dense(32, input_shape=(X_scaled.shape[1],), activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(y_encoded.shape[1], activation='softmax')  # softmax for multi-class classification
])
# Compile
predict_batted_balls.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
# Use early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# Fit
predict_batted_balls.fit(
    X_scaled, y_encoded,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop])
# Save
predict_batted_balls.save(os.path.join(model_path, "M01. Batted Balls", todaysdate, 'predict_batted_balls.keras'))

##### Predict

Predicted rates of events based on batted-ball data

In [ ]:
predictions = predict_batted_balls.predict(X_scaled)

prediction_df = pd.DataFrame(predictions, columns=encode_outcome.categories_[0])
prediction_df = prediction_df.add_suffix('_pred_batted')

prediction_df = pd.concat([complete_dataset.reset_index(drop=True), prediction_df], axis=1)

##### Evaluate

In [ ]:
# Adjust the number of rows and columns
n_events = len(events_list)
n_cols = 3
n_rows = (n_events + n_cols - 1) // n_cols  # Ceiling division

# Set square plots: each subplot is 5x5 inches
fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
axes = axes.flatten()

for i, event in enumerate(events_list):
    ax = axes[i]
    pred_col = f"{event}_pred_batted"
    
    if pred_col not in prediction_df.columns:
        continue

    # Bucket the predicted values into quantiles
    prediction_df['bucket'] = pd.qcut(prediction_df[pred_col], q=20, duplicates='drop')

    # Compute averages
    bucket_avg = prediction_df.groupby('bucket').agg(
        avg_pred=(pred_col, 'mean'),
        avg_actual=(event, 'mean')
    ).reset_index()

    # Plot
    ax.plot(bucket_avg['avg_pred'], label='Predicted')
    ax.plot(bucket_avg['avg_actual'], label='Actual')
    ax.set_title(f"{event.upper()} Prediction vs Actual")
    ax.set_xlabel("Quantile Bucket")
    ax.set_ylabel("Rate")
    ax.legend()
    ax.grid(True)

# Remove extra axes if any
for j in range(n_events, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()